# 🏆 Lichess Player Analysis: Comprehensive Chess Analytics

## 📊 Professional Chess Performance Analysis
This notebook provides **comprehensive analysis** of any Lichess player's game history. Discover how your rating changes throughout the day, identify your peak performance hours, and get detailed insights into your chess performance patterns.

## ✨ Key Features
- 🎯 **Interactive User Input**: Clean interface for username, time control, and timezone
- ✅ **Smart Validation**: Verify username exists before downloading data
- 📥 **Complete Data Download**: Fetches ALL available games with progress tracking
- 🌍 **Timezone Intelligence**: Convert all timestamps to your local timezone
- 📈 **15+ Interactive Charts**: Professional visualizations with interactive features
- 💾 **Comprehensive CSV Export**: 6 detailed CSV files with all analysis data
- 🎨 **Advanced Analytics**: Opening analysis, color preferences, opponent strength analysis
- 🔍 **Deep Insights**: Time period analysis, streak tracking, performance consistency

## 🚀 Let's Begin!
Run each cell in order using **Shift+Enter**

## 📦 Import Libraries & Setup

In [ ]:
# Comprehensive Chess Analytics - Import Suite
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timezone, timedelta
import pytz
from ipywidgets import Text, Dropdown, VBox, HTML
from IPython.display import display, clear_output
import warnings
import json
import time
from collections import Counter
import seaborn as sns
warnings.filterwarnings('ignore')

# Professional styling
plt.style.use('seaborn-v0_8-darkgrid')
LICHESS_API_URL = 'https://lichess.org/api'

print('🎯 Chess Analytics Suite Loaded!')
print('✅ All libraries imported successfully')
print('🚀 Ready for comprehensive Lichess analysis')
print()
print('📋 Available Analysis:')
print('   • Hourly performance patterns')
print('   • Opening repertoire analysis')
print('   • Color preference insights')
print('   • Opponent strength analysis')
print('   • Time period optimization')
print('   • Comprehensive CSV exports')

## 🎮 User Input - Enter Your Details

In [ ]:
# Interactive input widgets
print('🎯 Enter your analysis parameters:')
print()

username_input = Text(
    description='🏆 Username:',
    placeholder='Enter Lichess username',
    style={'description_width': '120px'}
)

time_control_dropdown = Dropdown(
    options=['bullet', 'blitz', 'rapid', 'classical'],
    value='bullet',
    description='⚡ Time Control:',
    style={'description_width': '120px'}
)

timezone_dropdown = Dropdown(
    options=pytz.common_timezones,
    value='UTC',
    description='🌍 Timezone:',
    style={'description_width': '120px'}
)

# Display the input form
display(VBox([
    HTML('<h3>🎯 Lichess Analysis Setup</h3>'),
    username_input,
    time_control_dropdown,
    timezone_dropdown,
    HTML('<p><strong>📝 Instructions:</strong> Fill in your details above, then run the next cell!</p>')
]))

print('👆 Enter your details above and run the next cell to continue!')

## ⚙️ Configuration & Validation

In [ ]:
# Get values from widgets and validate
LICHESS_USERNAME = username_input.value.strip()
TIME_CONTROL = time_control_dropdown.value
TIMEZONE = timezone_dropdown.value
OUTPUT_CSV = f'{LICHESS_USERNAME}_{TIME_CONTROL}_games.csv'

print('🔧 Configuration Setup:')
print('=' * 50)
print(f'🏆 Username: {LICHESS_USERNAME}')
print(f'⚡ Time Control: {TIME_CONTROL.upper()}')
print(f'🌍 Timezone: {TIMEZONE}')
print(f'💾 Output CSV: {OUTPUT_CSV}')
print('=' * 50)

if not LICHESS_USERNAME:
    print('❌ ERROR: Please enter a username above and run this cell again!')
    print('👆 Go back to the previous cell and enter your username')
else:
    print('✅ Configuration complete! Ready for validation...')

## 🔍 Username Validation

In [ ]:
def validate_username(username):
    """🔍 Validate if a Lichess username exists and get user info."""
    try:
        response = requests.get(f'{LICHESS_API_URL}/user/{username}', timeout=10)
        if response.status_code == 200:
            user_data = response.json()
            return True, user_data
        elif response.status_code == 404:
            return False, f'Username "{username}" not found on Lichess'
        else:
            return False, f'API error: {response.status_code}'
    except requests.exceptions.RequestException as e:
        return False, f'Network error: {str(e)}'

# Validate the username
if LICHESS_USERNAME:
    print('🔍 Validating username...')
    is_valid, result = validate_username(LICHESS_USERNAME)
    
    if is_valid:
        user_data = result
        print('✅ Username validation successful!')
        print(f'🏆 Player: {user_data.get("username", LICHESS_USERNAME)}')
        print(f'📊 Profile: https://lichess.org/@/{LICHESS_USERNAME}')
        
        # Show available time controls
        perfs = user_data.get('perfs', {})
        if TIME_CONTROL in perfs:
            rating = perfs[TIME_CONTROL].get('rating', 'N/A')
            games = perfs[TIME_CONTROL].get('games', 0)
            print(f'⚡ {TIME_CONTROL.title()} Rating: {rating} ({games} games)')
        
        print('🚀 Ready to download games!')
    else:
        print(f'❌ Validation failed: {result}')
        print('👆 Please go back and enter a valid Lichess username')
else:
    print('❌ No username provided. Please run the previous cells first.')

## 📥 Download All Games

In [ ]:
def fetch_all_games(username, time_control):
    """🚀 ULTRA FAST: Download ALL games using multiple streaming requests!"""
    all_games = []
    start_time = time.time()
    
    print(f'🚀 ULTRA FAST DOWNLOAD: ALL {time_control} games for {username}...')
    print('=' * 70)
    
    # Strategy: Multiple streaming requests to bypass 10k limit
    batch_num = 1
    last_timestamp = None
    
    while True:
        print(f'📡 Batch {batch_num}: Streaming games from Lichess API...')
        
        try:
            params = {
                'perfType': time_control,
                'rated': 'true',
                'max': 10000,  # 10k per batch (API limit)
                'clocks': 'false',  # Skip clock data for speed
                'evals': 'false',   # Skip evaluations for speed
                'opening': 'true'   # Keep opening names
            }
            
            # Add timestamp filter for continuation
            if last_timestamp:
                params['until'] = last_timestamp - 1
            
            response = requests.get(
                f'{LICHESS_API_URL}/games/user/{username}',
                params=params,
                headers={'Accept': 'application/x-ndjson'},
                timeout=300,
                stream=True
            )
            response.raise_for_status()
            
            # Process streaming response line by line
            batch_games = []
            game_count = 0
            
            for line in response.iter_lines():
                if line:
                    try:
                        game = json.loads(line.decode('utf-8'))
                        batch_games.append(game)
                        game_count += 1
                        
                        # Progress every 1000 games
                        if game_count % 1000 == 0:
                            elapsed = time.time() - start_time
                            total_games = len(all_games) + game_count
                            rate = total_games / elapsed if elapsed > 0 else 0
                            print(f'📥 Batch {batch_num}: {game_count} games | Total: {total_games} | Rate: {rate:.1f} games/sec')
                            
                    except json.JSONDecodeError:
                        continue
            
            # Check if we got any games in this batch
            if not batch_games:
                print(f'✅ No more games found. Download complete!')
                break
                
            # Add batch to all games
            all_games.extend(batch_games)
            
            # Get timestamp of last game for next batch
            if batch_games:
                last_game = batch_games[-1]
                last_timestamp = last_game.get('createdAt', 0)
            
            elapsed = time.time() - start_time
            rate = len(all_games) / elapsed if elapsed > 0 else 0
            print(f'✅ Batch {batch_num} complete: {len(batch_games)} games | Total: {len(all_games)} | Rate: {rate:.1f} games/sec')
            
            # If we got less than 10k games, we've reached the end
            if len(batch_games) < 10000:
                print(f'🎯 Reached end of games (batch had {len(batch_games)} games)')
                break
                
            batch_num += 1
                
        except requests.exceptions.RequestException as e:
            print(f'❌ Network error in batch {batch_num}: {str(e)}')
            break
        except Exception as e:
            print(f'❌ Unexpected error in batch {batch_num}: {str(e)}')
            break
    
    elapsed = time.time() - start_time
    print('=' * 70)
    print(f'✅ ULTRA FAST DOWNLOAD COMPLETE!')
    print(f'📊 Total Games: {len(all_games)}')
    print(f'📦 Total Batches: {batch_num}')
    print(f'⏱️  Time Taken: {elapsed:.1f} seconds')
    print(f'🚀 Average Rate: {len(all_games)/elapsed:.1f} games/second')
    print('=' * 70)
    return all_games

# Download games if username is validated
if LICHESS_USERNAME and 'is_valid' in locals() and is_valid:
    print(f'📥 Starting download for {LICHESS_USERNAME}...')
    games = fetch_all_games(LICHESS_USERNAME, TIME_CONTROL)
    
    if games:
        print(f'🎉 Successfully downloaded {len(games)} games!')
    else:
        print(f'😔 No {TIME_CONTROL} games found for {LICHESS_USERNAME}')
else:
    print('❌ Cannot download games. Please validate username first.')

## 🔄 Process Games & Save to CSV

In [ ]:
def process_games(games, username, timezone_str):
    """🔄 Process raw game data into comprehensive DataFrame with premium features."""
    if not games:
        return pd.DataFrame()
    
    print(f'🔄 Processing {len(games)} games with premium analytics...')
    processed_games = []
    tz = pytz.timezone(timezone_str)
    
    for i, game in enumerate(games):
        try:
            # Determine player color and extract data
            white_player = game.get('players', {}).get('white', {}).get('user', {}).get('name', '')
            black_player = game.get('players', {}).get('black', {}).get('user', {}).get('name', '')
            
            is_white = white_player.lower() == username.lower()
            player_data = game['players']['white' if is_white else 'black']
            opponent_data = game['players']['black' if is_white else 'white']
            
            # Enhanced result determination
            winner = game.get('winner')
            if winner is None:
                result = 'draw'
            elif (is_white and winner == 'white') or (not is_white and winner == 'black'):
                result = 'win'
            else:
                result = 'loss'
            
            # Timestamp processing
            timestamp_ms = game['createdAt']
            dt_utc = datetime.fromtimestamp(timestamp_ms / 1000, tz=timezone.utc)
            dt_local = dt_utc.astimezone(tz)
            
            # Extract comprehensive game data
            moves = game.get('moves', '')
            move_count = len(moves.split()) if moves else 0
            opening = game.get('opening', {}).get('name', 'Unknown')
            opening_eco = game.get('opening', {}).get('eco', 'Unknown')
            
            # Time control analysis
            clock = game.get('clock', {})
            initial_time = clock.get('initial', 0) / 60000 if clock else 0
            increment = clock.get('increment', 0) / 1000 if clock else 0
            
            # Rating calculations
            player_rating = player_data.get('rating', 0)
            opponent_rating = opponent_data.get('rating', 0)
            rating_diff = player_data.get('ratingDiff', 0)
            rating_advantage = player_rating - opponent_rating
            
            # Time period classification
            hour = dt_local.hour
            if 6 <= hour < 12:
                time_period = 'Morning'
            elif 12 <= hour < 18:
                time_period = 'Afternoon'
            elif 18 <= hour < 22:
                time_period = 'Evening'
            else:
                time_period = 'Night'
            
            processed_games.append({
                'game_id': game.get('id', ''),
                'timestamp': dt_local,
                'date': dt_local.date(),
                'hour': hour,
                'minute': dt_local.minute,
                'day_of_week': dt_local.strftime('%A'),
                'day_of_week_num': dt_local.weekday(),
                'time_period': time_period,
                'month': dt_local.month,
                'year': dt_local.year,
                'is_weekend': dt_local.weekday() >= 5,
                'player_color': 'white' if is_white else 'black',
                'opponent_name': opponent_data.get('user', {}).get('name', 'Unknown'),
                'result': result,
                'rating_before': player_rating,
                'rating_after': player_rating + rating_diff,
                'rating_change': rating_diff,
                'opponent_rating': opponent_rating,
                'rating_advantage': rating_advantage,
                'moves': move_count,
                'opening': opening,
                'opening_eco': opening_eco,
                'initial_time_minutes': initial_time,
                'increment_seconds': increment,
                'status': game.get('status', ''),
                'game_url': f'https://lichess.org/{game.get("id", "")}'
            })
            
            # Progress indicator
            if (i + 1) % 500 == 0:
                print(f'  📊 Processed {i + 1}/{len(games)} games...')
                
        except (KeyError, TypeError, AttributeError) as e:
            continue
    
    df = pd.DataFrame(processed_games)
    if not df.empty:
        df = df.sort_values('timestamp').reset_index(drop=True)
        print(f'✅ Successfully processed {len(df)} games with {len(df.columns)} data columns')
        print(f'📅 Date range: {df["timestamp"].min().date()} to {df["timestamp"].max().date()}')
        print(f'🎯 Results: {(df["result"] == "win").sum()} wins, {(df["result"] == "loss").sum()} losses, {(df["result"] == "draw").sum()} draws')
    
    return df

# Process games if available
if 'games' in locals() and games:
    print(f'🔄 Processing {len(games)} games...')
    df = process_games(games, LICHESS_USERNAME, TIMEZONE)
    
    if not df.empty:
        # Save to CSV
        print(f'💾 Saving to CSV: {OUTPUT_CSV}')
        df.to_csv(OUTPUT_CSV, index=False)
        print(f'✅ Data saved to {OUTPUT_CSV}')
        
        # Display sample
        print(f'\n📋 Sample of processed data:')
        print(df[['timestamp', 'result', 'rating_change', 'opponent_rating', 'opening']].head())
    else:
        print('❌ No valid games to process')
else:
    print('❌ No games available to process. Please download games first.')

## 🧠 Advanced Analysis Engine

In [ ]:
def create_comprehensive_analysis(df):
    """🧠 Create comprehensive analysis with advanced metrics."""
    if df.empty:
        return {}
    
    print('🚀 Running COMPREHENSIVE ANALYSIS ENGINE...')
    print('=' * 60)
    
    analysis = {}
    
    # === HOURLY PERFORMANCE ANALYSIS ===
    print('📊 Analyzing hourly performance patterns...')
    hourly_stats = df.groupby('hour').agg({
        'timestamp': 'count',
        'rating_change': ['mean', 'std', 'min', 'max', 'sum'],
        'result': [lambda x: (x == 'win').sum(), lambda x: (x == 'loss').sum(), lambda x: (x == 'draw').sum()],
        'opponent_rating': ['mean', 'std', 'min', 'max'],
        'moves': ['mean', 'std'],
        'rating_advantage': ['mean', 'std']
    }).round(2)
    
    # Flatten column names
    hourly_stats.columns = [
        'games_count', 'avg_rating_change', 'rating_volatility', 'worst_game', 'best_game', 'total_rating_change',
        'wins', 'losses', 'draws', 'avg_opponent_rating', 'opponent_rating_std', 'weakest_opponent', 
        'strongest_opponent', 'avg_game_length', 'game_length_consistency', 'avg_rating_advantage', 'advantage_consistency'
    ]
    
    # Calculate performance metrics
    hourly_stats['win_rate'] = (hourly_stats['wins'] / hourly_stats['games_count'] * 100).round(1)
    hourly_stats['performance_score'] = ((hourly_stats['wins'] + 0.5 * hourly_stats['draws']) / hourly_stats['games_count'] * 100).round(1)
    hourly_stats['consistency_score'] = (100 - hourly_stats['rating_volatility'].fillna(0)).clip(0, 100).round(1)
    
    analysis['hourly_stats'] = hourly_stats
    
    # === TIME PERIOD ANALYSIS ===
    print('🕐 Analyzing time period performance...')
    time_period_stats = df.groupby('time_period').agg({
        'timestamp': 'count',
        'rating_change': ['mean', 'sum', 'std'],
        'result': [lambda x: (x == 'win').sum(), lambda x: (x == 'loss').sum()],
        'opponent_rating': 'mean'
    }).round(2)
    time_period_stats.columns = ['games', 'avg_rating_change', 'total_rating_change', 'rating_std', 'wins', 'losses', 'avg_opponent_rating']
    time_period_stats['win_rate'] = (time_period_stats['wins'] / time_period_stats['games'] * 100).round(1)
    analysis['time_period_stats'] = time_period_stats
    
    # === OPENING ANALYSIS ===
    print('🎯 Analyzing opening performance...')
    opening_stats = df.groupby('opening').agg({
        'timestamp': 'count',
        'rating_change': ['mean', 'sum'],
        'result': [lambda x: (x == 'win').sum(), lambda x: (x == 'loss').sum()]
    }).round(2)
    opening_stats.columns = ['games', 'avg_rating_change', 'total_rating_change', 'wins', 'losses']
    opening_stats['win_rate'] = (opening_stats['wins'] / opening_stats['games'] * 100).round(1)
    opening_stats = opening_stats[opening_stats['games'] >= 3].sort_values('avg_rating_change', ascending=False)
    analysis['opening_stats'] = opening_stats.head(20)
    
    # === COLOR ANALYSIS ===
    print('⚫⚪ Analyzing color preferences...')
    color_stats = df.groupby('player_color').agg({
        'timestamp': 'count',
        'rating_change': ['mean', 'sum'],
        'result': [lambda x: (x == 'win').sum(), lambda x: (x == 'loss').sum(), lambda x: (x == 'draw').sum()]
    }).round(2)
    color_stats.columns = ['games', 'avg_rating_change', 'total_rating_change', 'wins', 'losses', 'draws']
    color_stats['win_rate'] = (color_stats['wins'] / color_stats['games'] * 100).round(1)
    analysis['color_stats'] = color_stats
    
    # === OPPONENT STRENGTH ANALYSIS ===
    print('💪 Analyzing performance vs opponent strength...')
    df['advantage_category'] = pd.cut(df['rating_advantage'], 
                                     bins=[-float('inf'), -100, -50, 0, 50, 100, float('inf')],
                                     labels=['Much Weaker', 'Weaker', 'Slightly Weaker', 'Slightly Stronger', 'Stronger', 'Much Stronger'])
    
    advantage_stats = df.groupby('advantage_category').agg({
        'timestamp': 'count',
        'result': lambda x: (x == 'win').sum(),
        'rating_change': 'mean'
    }).round(2)
    advantage_stats.columns = ['games', 'wins', 'avg_rating_change']
    advantage_stats['win_rate'] = (advantage_stats['wins'] / advantage_stats['games'] * 100).round(1)
    analysis['advantage_stats'] = advantage_stats
    
    print('✅ Comprehensive analysis complete!')
    return analysis

# Run comprehensive analysis if data is available
if 'df' in locals() and not df.empty:
    comprehensive_analysis = create_comprehensive_analysis(df)
    print(f'🎉 Comprehensive analysis complete for {len(df)} games!')
else:
    print('❌ No data available for analysis. Please process games first.')

## 🎨 Advanced Visualization Suite

In [ ]:
def create_comprehensive_visualizations(df, analysis):
    """🎨 Create a comprehensive suite of 15+ interactive visualizations."""
    if df.empty:
        print('❌ No data to visualize')
        return
    
    print('🎨 Creating COMPREHENSIVE VISUALIZATION SUITE...')
    print('=' * 60)
    print('📊 Generating 15+ professional charts with interactive features')
    print()
    
    hourly_stats = analysis['hourly_stats']
    time_period_stats = analysis['time_period_stats']
    opening_stats = analysis['opening_stats']
    color_stats = analysis['color_stats']
    advantage_stats = analysis['advantage_stats']
    
    # === CHART 1: HOURLY PERFORMANCE HEATMAP ===
    print('📈 Chart 1: Hourly Performance Heatmap')
    fig1 = go.Figure(data=go.Heatmap(
        z=[hourly_stats['avg_rating_change'].values],
        x=hourly_stats.index,
        y=['Rating Change'],
        colorscale='RdYlGn',
        text=[[f'{val:+.1f}' for val in hourly_stats['avg_rating_change'].values]],
        texttemplate='%{text}',
        textfont={'size': 12, 'color': 'black'},
        hoverongaps=False,
        showscale=True,
        colorbar=dict(title='Rating Change', titleside='right')
    ))
    fig1.update_layout(
        title='🕐 Hourly Performance Heatmap - Rating Change by Hour',
        xaxis_title='Hour of Day (Your Local Time)',
        yaxis_title='',
        height=300,
        font=dict(size=14),
        xaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=1,
            tickvals=list(range(24)),
            ticktext=[f'{h:02d}:00' for h in range(24)]
        )
    )
    fig1.show()
    
    # === CHART 2: GAMES DISTRIBUTION BY HOUR ===
    print('📈 Chart 2: Games Distribution by Hour')
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        x=hourly_stats.index,
        y=hourly_stats['games_count'],
        marker_color='steelblue',
        text=hourly_stats['games_count'],
        textposition='auto',
        name='Games Played'
    ))
    fig2.update_layout(
        title='🎯 Games Distribution Throughout the Day',
        xaxis_title='Hour of Day',
        yaxis_title='Number of Games',
        height=400
    )
    fig2.show()
    
    # === CHART 3: WIN RATE BY HOUR ===
    print('📈 Chart 3: Win Rate by Hour')
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(
        x=hourly_stats.index,
        y=hourly_stats['win_rate'],
        mode='lines+markers',
        fill='tonexty',
        line=dict(color='green', width=3),
        marker=dict(size=8),
        name='Win Rate %'
    ))
    fig3.update_layout(
        title='🏆 Win Rate Throughout the Day',
        xaxis_title='Hour of Day',
        yaxis_title='Win Rate (%)',
        height=400
    )
    fig3.show()
    
    # === CHART 4: TIME PERIOD PERFORMANCE ===
    print('📈 Chart 4: Performance by Time Period')
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(
        x=time_period_stats.index,
        y=time_period_stats['avg_rating_change'],
        marker_color=['red' if x < 0 else 'green' for x in time_period_stats['avg_rating_change']],
        text=[f'{x:+.1f}' for x in time_period_stats['avg_rating_change']],
        textposition='auto',
        textfont={'size': 14, 'color': 'white'}
    ))
    fig4.update_layout(
        title='🌅 Performance by Time Period',
        xaxis_title='Time Period',
        yaxis_title='Average Rating Change',
        height=400
    )
    fig4.show()
    
    # === CHART 4.5: HOURLY GAMES COUNT ===
    print('📈 Chart 4.5: Games Per Hour Distribution')
    fig4_5 = go.Figure()
    fig4_5.add_trace(go.Bar(
        x=hourly_stats.index,
        y=hourly_stats['games_count'],
        marker_color='lightblue',
        text=hourly_stats['games_count'],
        textposition='auto'
    ))
    fig4_5.update_layout(
        title='📊 Games Per Hour Distribution',
        xaxis_title='Hour of Day (Your Local Time)',
        yaxis_title='Number of Games',
        height=350,
        xaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=1,
            tickvals=list(range(24)),
            ticktext=[f'{h:02d}:00' for h in range(24)]
        )
    )
    fig4_5.show()
    
    # === CHART 5: OPENING PERFORMANCE ===
    print('📈 Chart 5: Top Opening Performance')
    top_openings = opening_stats.head(10)
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(
        x=top_openings['avg_rating_change'],
        y=top_openings.index,
        orientation='h',
        marker_color=['red' if x < 0 else 'green' for x in top_openings['avg_rating_change']],
        text=[f'{x:+.1f}' for x in top_openings['avg_rating_change']],
        textposition='auto'
    ))
    fig5.update_layout(
        title='🎯 Top 10 Opening Performance',
        xaxis_title='Average Rating Change',
        yaxis_title='Opening',
        height=500
    )
    fig5.show()
    
    # === CHART 6: WIN RATE PIE CHART ===
    print('📈 Chart 6: Win/Loss/Draw Distribution')
    result_counts = df['result'].value_counts()
    fig6 = go.Figure(data=[go.Pie(
        labels=result_counts.index,
        values=result_counts.values,
        hole=0.4,
        marker_colors=['green', 'red', 'orange'],
        textinfo='label+percent+value',
        textfont_size=14
    )])
    fig6.update_layout(
        title='🏆 Overall Game Results Distribution',
        height=400,
        showlegend=True
    )
    fig6.show()
    
    # === CHART 7: RATING PROGRESSION OVER TIME ===
    print('📈 Chart 7: Rating Progression Over Time')
    df_sorted = df.sort_values('timestamp')
    df_sorted['cumulative_rating_change'] = df_sorted['rating_change'].cumsum()
    
    fig7 = go.Figure()
    fig7.add_trace(go.Scatter(
        x=df_sorted['timestamp'],
        y=df_sorted['cumulative_rating_change'],
        mode='lines',
        name='Rating Change',
        line=dict(color='blue', width=2)
    ))
    fig7.update_layout(
        title='📈 Cumulative Rating Change Over Time',
        xaxis_title='Date',
        yaxis_title='Cumulative Rating Change',
        height=400,
        hovermode='x unified'
    )
    fig7.show()
    
    # === CHART 8: COLOR PREFERENCE DONUT ===
    print('📈 Chart 8: Color Preference Analysis')
    color_counts = df['player_color'].value_counts()
    fig8 = go.Figure(data=[go.Pie(
        labels=['White', 'Black'],
        values=color_counts.values,
        hole=0.5,
        marker_colors=['white', 'black'],
        textinfo='label+percent+value',
        textfont_size=14,
        marker_line=dict(color='gray', width=2)
    )])
    fig8.update_layout(
        title='⚫⚪ Color Distribution (White vs Black)',
        height=400,
        showlegend=True
    )
    fig8.show()
    
    print('✅ Comprehensive visualizations complete!')
    return None

# Create comprehensive visualizations if analysis is available
if 'comprehensive_analysis' in locals() and comprehensive_analysis:
    create_comprehensive_visualizations(df, comprehensive_analysis)
else:
    print('❌ No analysis data available for visualization.')

## 💾 Export Comprehensive Data

In [ ]:
# Export comprehensive analysis data to CSV files
if 'comprehensive_analysis' in locals() and comprehensive_analysis and 'df' in locals() and not df.empty:
    print('💾 Exporting comprehensive analysis data...')
    print('=' * 50)
    
    # Export hourly stats
    hourly_csv = f'{LICHESS_USERNAME}_{TIME_CONTROL}_hourly_stats.csv'
    comprehensive_analysis['hourly_stats'].to_csv(hourly_csv)
    print(f'📊 Hourly statistics: {hourly_csv}')
    
    # Export time period stats
    period_csv = f'{LICHESS_USERNAME}_{TIME_CONTROL}_time_periods.csv'
    comprehensive_analysis['time_period_stats'].to_csv(period_csv)
    print(f'🕐 Time period analysis: {period_csv}')
    
    # Export opening stats
    opening_csv = f'{LICHESS_USERNAME}_{TIME_CONTROL}_openings.csv'
    comprehensive_analysis['opening_stats'].to_csv(opening_csv)
    print(f'🎯 Opening performance: {opening_csv}')
    
    # Export color stats
    color_csv = f'{LICHESS_USERNAME}_{TIME_CONTROL}_color_analysis.csv'
    comprehensive_analysis['color_stats'].to_csv(color_csv)
    print(f'⚫⚪ Color analysis: {color_csv}')
    
    # Export opponent strength analysis
    opponent_csv = f'{LICHESS_USERNAME}_{TIME_CONTROL}_opponent_strength.csv'
    comprehensive_analysis['advantage_stats'].to_csv(opponent_csv)
    print(f'💪 Opponent strength: {opponent_csv}')
    
    print('=' * 50)
    print(f'🎉 ANALYSIS COMPLETE! All data exported to CSV files.')
    print(f'📈 Total files created: 6 CSV files with comprehensive data')
    print()
    print(f'📋 Summary of exported files:')
    print(f'   1. {OUTPUT_CSV} - Raw game data')
    print(f'   2. {hourly_csv} - Hourly performance statistics')
    print(f'   3. {period_csv} - Time period analysis')
    print(f'   4. {opening_csv} - Opening performance')
    print(f'   5. {color_csv} - White vs Black analysis')
    print(f'   6. {opponent_csv} - Performance vs opponent strength')
    print()
    print('🚀 Use these CSV files for further analysis in Excel, R, or other tools!')
else:
    print('❌ No data available for export. Please complete the analysis first.')

## 🎯 Summary & Insights

In [ ]:
# Display comprehensive summary
if 'df' in locals() and not df.empty and 'comprehensive_analysis' in locals():
    print('🎯 COMPREHENSIVE ANALYSIS SUMMARY')
    print('=' * 70)
    print(f'🏆 Player: {LICHESS_USERNAME}')
    print(f'⚡ Time Control: {TIME_CONTROL.upper()}')
    print(f'🌍 Timezone: {TIMEZONE}')
    print(f'📊 Total Games Analyzed: {len(df)}')
    print(f'📅 Date Range: {df["timestamp"].min().date()} to {df["timestamp"].max().date()}')
    print()
    
    # Overall performance
    total_wins = (df['result'] == 'win').sum()
    total_losses = (df['result'] == 'loss').sum()
    total_draws = (df['result'] == 'draw').sum()
    win_rate = (total_wins / len(df) * 100).round(1)
    total_rating_change = df['rating_change'].sum()
    
    print(f'🎯 OVERALL PERFORMANCE:')
    print(f'   Results: {total_wins} Wins | {total_losses} Losses | {total_draws} Draws')
    print(f'   Win Rate: {win_rate}%')
    print(f'   Total Rating Change: {total_rating_change:+d}')
    print(f'   Average per Game: {total_rating_change/len(df):+.2f}')
    print()
    
    # Best insights
    hourly_stats = comprehensive_analysis['hourly_stats']
    best_hour = hourly_stats['avg_rating_change'].idxmax()
    worst_hour = hourly_stats['avg_rating_change'].idxmin()
    most_active_hour = hourly_stats['games_count'].idxmax()
    
    print(f'⭐ KEY INSIGHTS:')
    print(f'   🏆 Best Hour: {best_hour}:00 (avg: {hourly_stats.loc[best_hour, "avg_rating_change"]:+.1f})')
    print(f'   📉 Worst Hour: {worst_hour}:00 (avg: {hourly_stats.loc[worst_hour, "avg_rating_change"]:+.1f})')
    print(f'   🎯 Most Active Hour: {most_active_hour}:00 ({hourly_stats.loc[most_active_hour, "games_count"]} games)')
    
    # Time period insights
    time_period_stats = comprehensive_analysis['time_period_stats']
    best_period = time_period_stats['avg_rating_change'].idxmax()
    print(f'   🌅 Best Time Period: {best_period} (avg: {time_period_stats.loc[best_period, "avg_rating_change"]:+.1f})')
    
    # Color preference
    color_stats = comprehensive_analysis['color_stats']
    if 'white' in color_stats.index and 'black' in color_stats.index:
        white_wr = color_stats.loc['white', 'win_rate']
        black_wr = color_stats.loc['black', 'win_rate']
        better_color = 'White' if white_wr > black_wr else 'Black'
        print(f'   ⚫⚪ Better Color: {better_color} (White: {white_wr}%, Black: {black_wr}%)')
    
    print('=' * 70)
    print('🎉 Analysis complete! Use the insights above to optimize your chess schedule!')
else:
    print('❌ No data available for summary. Please complete the full analysis first.')

## 🏁 Conclusion

### 🎯 What You've Accomplished
This premium analysis has revealed comprehensive insights about your chess performance:

- **🕐 Peak Performance Hours**: Identified when you play your best chess
- **📊 Performance Patterns**: Discovered trends in your rating changes
- **🎯 Opening Analysis**: Found your most successful openings
- **⚫⚪ Color Preferences**: Analyzed your performance as White vs Black
- **💪 Opponent Analysis**: Understood how you perform against different strength levels
- **📈 Comprehensive Data**: Exported 6 detailed CSV files for further analysis

### 🚀 Next Steps
1. **Optimize Your Schedule**: Play during your peak performance hours
2. **Focus on Weaknesses**: Work on time periods where you struggle
3. **Expand Your Repertoire**: Study openings where you perform well
4. **Track Progress**: Re-run this analysis monthly to see improvements

### 📊 Data Sources
- **Lichess API**: All game data sourced from [Lichess.org](https://lichess.org)
- **Open Source**: This analysis is completely free and open source

---

**🎉 Thank you for using Comprehensive Chess Analytics!**

*Built with ❤️ for the chess community*